<a href="https://colab.research.google.com/github/HilbertN/Redes_Neuronales/blob/main/RN15_ResidualConv2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
learning_rate = 0.01
epochs = 30
batch_size = 120
optimizer = 'rmsprop'

In [ ]:
!wget https://dagshub.com/jorgevc/Gatos-y-Perros/raw/24eaabd8ca81ed89f780138c1e3955ccd1f3d155/data/DogsCats.zip

--2023-10-06 16:21:44--  https://dagshub.com/jorgevc/Gatos-y-Perros/raw/24eaabd8ca81ed89f780138c1e3955ccd1f3d155/data/DogsCats.zip
Resolving dagshub.com (dagshub.com)... 35.186.200.224
Connecting to dagshub.com (dagshub.com)|35.186.200.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘DogsCats.zip’

DogsCats.zip            [   <=>              ] 545.56M  78.2MB/s    in 7.1s    

2023-10-06 16:21:52 (76.9 MB/s) - ‘DogsCats.zip’ saved [572061537]



In [ ]:
!unzip DogsCats.zip

In [ ]:
ih, iw = 150, 150 #tamano de la imagen
input_shape = (ih, iw, 3) #forma de la imagen: alto ancho y numero de canales

#train_dir = 'data/minitrain' #directorio de entrenamiento
#test_dir = 'data/minitest' #directorio de prueba
train_dir = 'train' #directorio de entrenamiento
test_dir = 'test' #directorio de prueba


num_class = 2 #cuantas clases
epochs = 30 #cuantas veces entrenar. En cada epoch hace una mejora en los parametros

batch_size = 50 #batch para hacer cada entrenamiento. Lee 50 'batch_size' imagenes antes de actualizar los parametros. Las carga a memoria
num_train = 20000 #numero de imagenes en train
num_test = 5000 #numero de imagenes en test


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


gentrain = ImageDataGenerator(rescale=1. / 255.) #indica que reescale cada canal con valor entre 0 y 1.


train = gentrain.flow_from_directory(train_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

gentest = ImageDataGenerator(rescale=1. / 255)

test = gentest.flow_from_directory(test_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
#########  Modelo

inputs = keras.Input(shape=input_shape)
x = layers.Conv2D(10, (3,3))(inputs)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

residual = x
x = layers.Conv2D(10, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)
residual=layers.Conv2D(10,1,strides=2)(residual)
x = layers.add([x, residual])

residual = x
x = layers.Conv2D(20,3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
residual=layers.Conv2D(20,4,strides=2)(residual) #quitar para ejemplificar que los tamaños deben coincidir
x = layers.add([x, residual])

x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()
############

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_49 (Conv2D)          (None, 148, 148, 10)         280       ['input_11[0][0]']            
                                                                                                  
 activation_10 (Activation)  (None, 148, 148, 10)         0         ['conv2d_49[0][0]']           
                                                                                                  
 max_pooling2d_30 (MaxPooli  (None, 74, 74, 10)           0         ['activation_10[0][0]']       
 ng2D)                                                                                      

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=test
    )

model.save("Conv2D.hf5")

Epoch 1/30
400/400 [==============================] - 345s 858ms/step - loss: 0.6308 - accuracy: 0.6407 - val_loss: 0.5468 - val_accuracy: 0.7244
Epoch 2/30
400/400 [==============================] - 336s 840ms/step - loss: 0.5138 - accuracy: 0.7488 - val_loss: 0.4871 - val_accuracy: 0.7608
Epoch 3/30
400/400 [==============================] - 326s 814ms/step - loss: 0.4340 - accuracy: 0.7994 - val_loss: 0.5008 - val_accuracy: 0.7658
Epoch 4/30
400/400 [==============================] - 325s 811ms/step - loss: 0.3663 - accuracy: 0.8361 - val_loss: 0.4762 - val_accuracy: 0.7850
Epoch 5/30
400/400 [==============================] - 342s 854ms/step - loss: 0.2908 - accuracy: 0.8775 - val_loss: 0.4835 - val_accuracy: 0.7870
Epoch 6/30
400/400 [==============================] - 336s 840ms/step - loss: 0.2137 - accuracy: 0.9159 - val_loss: 0.5525 - val_accuracy: 0.7858
Epoch 7/30
400/400 [==============================] - 335s 838ms/step - loss: 0.1483 - accuracy: 0.9439 - val_loss: 0.6232 -